In [2]:
from tqdm import tqdm
from PIL import Image
from io import BytesIO
from pathlib import Path
import requests
import pandas as pd

### Dataloader

In [27]:
root = 'https://oss.xiaoshanchongdie.com/items'
troot = Path('/nas/chenyi/datasets_nas/deploy_system/anomaly/20220902/')
# url = 'https://oss.xiaoshanchongdie.com/items/76c6d44e-300c-4f85-b8bb-46b87e88f456.jpg'

In [35]:
img_exist_df = pd.read_csv('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0823_deploy.csv')
img_new_df = pd.read_csv('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0902_deploy.csv')
img_new_df.rename(columns={'单品文件名': 'fname'}, inplace=True)
img_exist_df.rename(columns={'单品文件名': 'fname'}, inplace=True)
img_cross_df = img_new_df[~img_new_df['fname'].isin(img_exist_df['fname'])].copy(deep=True)
img_cross_df['spath'] = img_cross_df['fname'].apply(lambda x: f'{root}/{x}')

##### xlsx

In [13]:
xlsx_df = pd.read_excel('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0902_deploy.xlsx')
xlsx_df.to_csv('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0902_deploy.csv', index=False)

##### TXT

In [ ]:
txt_p = Path('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0823_deploy.txt')
with open(txt_p, 'r') as f:
    t = f.readlines()
img_df = pd.DataFrame()
img_df['fname'] = t
img_df['fname'] = img_df['fname'].apply(lambda x: x.strip())
img_df['spath'] = img_df['fname'].apply(lambda x: f'{root}/{x}')
len(img_df)

In [11]:
img_df.to_csv('/nas/chenyi/datasets_nas/deploy_system/csv/image_name_list/anomaly0823_deploy.csv', index=False)

### 数据下载

#### 全部数据下载

In [36]:
img_df = img_cross_df
img_df['id'] =  list(img_df.index)
img_df['folder'] = img_df['id'].apply(lambda x: int(x//500))
img_df['tpath'] = img_df.apply(lambda row: '{0}/{1}/{2}'.format(troot, row['folder'], row['fname']), axis=1)
show_root = Path(img_df['tpath'].values[0]).parents[2]
if not show_root.is_dir(): show_root.mkdir(parents=True)
!ls $show_root

In [37]:
for ind, row in tqdm(img_df.iterrows()):
    sp = row['spath']
    tp = Path(row['tpath'])
    if tp.is_file():
        continue
    try:
        req = requests.get(sp)
        img = Image.open(BytesIO(req.content))
        if not Path(tp).parent.is_dir():
            Path(tp).parent.mkdir(parents=True)
        img.save(tp)
    except:
        print(row['spath'])
        # print('\n')
    # break

5299it [19:32,  3.62it/s]

https://oss.xiaoshanchongdie.com/items/f23c1d01-2831-4dab-93b6-a138ff459b68.jpg


6218it [23:49, 15.47s/it]

https://oss.xiaoshanchongdie.com/items/98fddab7-db5a-4e01-8eda-afaa3c2f7bd4.jpg


7025it [26:53,  4.35it/s]


#### 部分数据下载

In [61]:
img_0_df = img_cross_df[~(img_cross_df['origin_type_id']==0)]
img_df = img_0_df.copy(deep=True)
len(img_0_df)

191

In [62]:
img_df['folder'] = 'origin_type_id0'

In [64]:
img_df['spath'] = img_df['path'].apply(lambda x: f'{root}/{x}')
# img_df['tpath'] = img_df['path'].apply(lambda x: f'{troot}/{x}')
img_df['tpath'] = img_df.apply(lambda row: '{0}/{1}/{2}_{3}{4}'.format(troot, row['folder'],Path(row['spath']).stem, row['type_id'], Path(row['spath']).suffix), axis=1)

In [90]:
save_df = img_df[:]
for ind, row in tqdm(save_df.iterrows()):
    sp = row['spath']
    tp = Path(row['tpath'])
    if tp.is_file():
        continue
    try:
        req = requests.get(sp)
        img = Image.open(BytesIO(req.content))
        if not Path(tp).parent.is_dir():
            Path(tp).parent.mkdir(parents=True)
        img.save(tp)
    except:
        print(row['spath'])
        # print('\n')
    # break

191it [00:28,  6.75it/s]


### 数据去重

In [60]:
import hashlib
import os

In [51]:
# img_df.to_csv('query_img_20220622.csv', index=False)

In [52]:
img_df.head(2)

,id,user_id,path,width,height,type_id,一级分类,category_id,二级分类,spath,tpath,folder
0,1,1,8435705c-421f-45dc-abd4-c1c23d38cbfb.jpg,690,1034,4,上装,0,NaN,https://oss.xiaoshanchongdie.com/items/8435705...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0
1,2,3,d943be5f-92aa-4017-abaf-4130621322df.png,2268,4032,7,连体装,0,NaN,https://oss.xiaoshanchongdie.com/items/d943be5...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0


In [61]:
# 使用python3.8及以上可以用此方法，写法更简洁。
def file_hash(file_path: str, hash_method) -> str:
    if not os.path.isfile(file_path):
        print('文件不存在。')
        return ''
    h = hash_method()
    with open(file_path, 'rb') as f:
        while b := f.read(8192):
            h.update(b)
    return h.hexdigest()

def file_sha256(file_path: str) -> str:
    return file_hash(file_path, hashlib.sha256)

In [56]:
imgs_list = img_df['tpath'].values
troot = '/nas/chenyi/datasets_nas/dada_det/data/wholebody0602/wholebody/'

In [ ]:
# for path in imgs_list:
#     src = path
#     dst = '/nas/chenyi/datasets_nas/dada_det/data/wholebody0602/wholebody/' + file_sha256(path) + '.jpg'
#     with open('log.txt', 'a', encoding='utf-8') as f:
#         f.write(f'{src} -> {dst}\n')
#         shutil.copy(src, dst)

In [ ]:
hash_list = []
for path in tqdm(imgs_list):
    hash_list.append(file_sha256(path))
    # break

In [66]:
img_df['hash'] = hash_list

In [71]:
len(hash_list), len(img_df), len(img_df.drop_duplicates('hash'))

(19465, 19465, 17436)

,id,user_id,path,width,height,type_id,一级分类,category_id,二级分类,spath,tpath,folder,hash,label_eng
0,1,1,8435705c-421f-45dc-abd4-c1c23d38cbfb.jpg,690,1034,4,上装,0,NaN,https://oss.xiaoshanchongdie.com/items/8435705...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0,2c4c0495ee10f7e3691249a1db5c6eeb7c2616099b6124...,NaN
1,2,3,d943be5f-92aa-4017-abaf-4130621322df.png,2268,4032,7,连体装,0,NaN,https://oss.xiaoshanchongdie.com/items/d943be5...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0,79d123e113ba021915f0296993729be66133fecbeb9592...,NaN
2,3,5,25fc8e21-e522-4dba-9438-4b7c0e86069d.jpg,960,444,4,上装,0,NaN,https://oss.xiaoshanchongdie.com/items/25fc8e2...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0,be3bdb0eefd5c1f7ab895db20ff210263441c9cbe10fcc...,NaN
3,4,3,bd2fab02-e026-47e3-ab35-bde7865a4eb0.png,2268,4032,4,上装,0,NaN,https://oss.xiaoshanchongdie.com/items/bd2fab0...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0,a6d8660a066b0173495db6a14eb5b660d31e9c7e46d9cb...,NaN
4,5,3,d616a39a-3f16-4bcc-9a12-e8ce745cc80a.png,2268,4032,5,裤子,0,NaN,https://oss.xiaoshanchongdie.com/items/d616a39...,/nas/chenyi/datasets_nas/deploy_sys/dadetv3/0/...,0,42f9a78bd178de188d6b095a22492f3ef20422613afa0c...,NaN


### 重新整理folder

In [ ]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [72]:
model_label_map = {
    10: 'shoe',  # 鞋子
    8: 'bag',  # 包包
    4: 'upperbody',  # 上装
    5: 'paints',  # 裤子
    6: 'skirt',  # 裙子
    7: 'wholebody',  # 连体装
}

In [73]:
# img_df['label_eng'] = img_df['category_id'].map(model_label_map)

In [94]:
troot = '/nas/chenyi/datasets_nas/deploy_sys/dadetv3_cls_bak'

In [95]:
img_df['new_path'] = img_df.apply(lambda row: '{0}/{1}/{2}/{3}.jpg'.format(troot, row['一级分类'], row['folder'], Path(row['spath']).stem), 
                                  axis=1)

In [96]:
df = img_df.drop_duplicates('hash')

In [97]:
import shutil
for ind, row in tqdm(df.iterrows()):
    sp = row['tpath']
    tp = row['new_path']
    if not Path(tp).parent.is_dir():
        Path(tp).parent.mkdir(parents=True)
    if Path(sp).is_file() and (not Path(tp).is_file()):
        shutil.copyfile(sp, tp)
    # break

17436it [00:37, 470.57it/s]


In [99]:
# !mkdir -p /nas/chenyi/workspace/myPyModule/dataset/trainset_generate/

In [100]:
# !cp -r /home/chenyi/workspace/myPyModule99/dataset/trainset_generate/data/ \
#         /nas/chenyi/workspace/myPyModule/dataset/trainset_generate/